In [1]:
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertForSequenceClassification, BertConfig
import torch
import re
from collections import OrderedDict
import numpy as np
import json
import datetime
import predict
import info
import handle

In [2]:
"""
这里是模型的调用测试
参考这里的语句调用模型！
"""


# * 加载分类器 model1 用于语句分类  | model2 用于岗位匹配 *
# model_name = 'hfl/chinese-roberta-wwm-ext'
model_name = './models/bert-roberta'
tokenizer1 = BertTokenizer.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name)
config.num_hidden_layers = 2
config.num_labels = 7
model1 = BertForSequenceClassification.from_pretrained(
    model_name, config=config)
model1.load_state_dict(torch.load(
    'models/Roberta.pt', map_location=torch.device('cpu')), strict=False)
model1.eval()


tokenizer2 = BertTokenizer.from_pretrained(model_name)
config2 = BertConfig.from_pretrained(model_name)
config2.num_hidden_layers = 2
config2.num_labels = 11
model2 = BertForSequenceClassification.from_pretrained(
    model_name, config=config2)
model2.load_state_dict(torch.load(
    'models/Job.pt', map_location=torch.device('cpu')), strict=False)
model2.eval()


# 加载 NER
tokenizer3 = AutoTokenizer.from_pretrained(model_name)
pretrained3 = AutoModel.from_pretrained(model_name)


# 定义下游模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tuning = False
        self.pretrained = None

        self.rnn = torch.nn.GRU(768, 768, batch_first=True)
        self.fc = torch.nn.Linear(768, 8)

    def forward(self, inputs):
        if self.tuning:
            out = self.pretrained(**inputs).last_hidden_state
        else:
            with torch.no_grad():
                out = pretrained3(**inputs).last_hidden_state

        out, _ = self.rnn(out)
        out = self.fc(out).softmax(dim=2)
        return out


model3 = torch.load('models/Ner.pt', map_location=torch.device('cpu'))
model3.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./models/bert-roberta and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./models/bert-roberta and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model(
  (rnn): GRU(768, 768, batch_first=True)
  (fc): Linear(in_features=768, out_features=8, bias=True)
  (pretrained): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, ou

In [3]:
def analysis(sentences):
    """
    文本的预处理、使用模型进行分类和命名实体识别，以及根据模型输出和特定逻辑组织最终的分析结果
    :param sentences:要分析的简历文本
    :return:将分析结果整理为一个包含多个维度信息的字典，如基本信息、工作经验、技能等，并以JSON格式返回
    """
    # sentences = list(OrderedDict.fromkeys(sentences))
    # ? sequence 为最初分类序列 经过上下文纠正算法得到最终分类结果 data
    sequence = []
    data = [[] for _ in range(7)]
    for s in sentences:
        p = predict.label_predict(s, tokenizer1, model1)
        s = re.sub(r'[\s\t]{2,}', ' ', s.strip())
        sequence.append([s, p])

    # 上下文纠正算法 纠正特殊标签6 这里有两种可能 6,2->6,1 或 6,2->2,2
    for i in range(0, len(sequence)-1):
        if sequence[i][1] == 6 and sequence[i+1][1] == 2:
            job_obj_patter = '|'.join(info.job_obj_keywords())
            matches = re.findall(job_obj_patter, sequence[i][0])
            if matches:
                # 6,2->6,1
                sequence[i+1][1] = 1
            else:
                # 6,2->2,2
                sequence[i][1] = 2

    # 至此分类完成
    for s in sequence:
        data[s[1]].append(s[0])
    # 对基本信息单独NER+正则匹配
    basic_data = {
        'name': '',
        'birth': '',
        'age': 0,
        'tel': '',
        'email': '',
        'college': [],
        'loc': [],
        'edu': ''
    }

    tag = {
        'edu_tag': [],
        'loc_tag': '',
        'experience_tag': [],
        'ability': [],
        'total_work_time': ''
    }

    total_data = {
        'basic_data': basic_data,
        'job_obj': data[1],
        'experience': data[2],
        'award': list(OrderedDict.fromkeys(data[3])),
        'ability': list(OrderedDict.fromkeys(data[4])),
        'job_fit': [],
        'tag': tag,
        'score': 0,
        'custom_content': {
            'money_obj': '',
            'self_desc': [],
            'self_tag': []
        }
    }
    handle.handle_basedata(data, basic_data, total_data, tokenizer3, model3)
    handle.handle_job_obj(total_data)
    handle.handle_experience(total_data, tokenizer3, model3)
    handle.handle_ability(total_data)
    handle.handle_job_fit(total_data, tokenizer2, model2)

    return json.dumps(total_data, ensure_ascii=False)

In [4]:
sentences1 = ['张吉惟','EXCEL','PPT','WORD','AI','PS','个人技能','出生年月：1998.11','性 别：女','籍 贯：北京海淀区','学 历：本科','政治面貌：党员','身 高：178cm','体 重：50kg','工作经历','自我评价','个人爱好','本人性格开朗、稳重、有活力，待人热情、真诚。有较强的组织能力、团体协作精神，较好的社交能力，善于处理各种人际关系。能迅速的适应各种环境，并融合其中。能把企业当作家庭，企业的财富就是我的财富，在努力为企业服务的过程中实现自身价值。','zhangjiwei@163.com','138 9999 9999','求职意向：行政管理类','基本信息','2016\t全国大学生英语竞赛一等奖','2017\t国家级一等奖学金','2018\t大学英语6级证书（CET-6）','奖励证书','教育背景','2015-2019\t国际经济与贸易','北京师范大学','政治经济学、西方经济学、国际经济学、计量经济学、世界经济概论、国际贸易理论与实务、国际金融、国际结算。','2022.3至今\t部长助理','北京猫眼机械有限公司','质管部部长助理.负责票证车间生产质量检查,控制,兼任企业内审员,推行ISO9001:2008及ISO14000体系并年审。','2021.2-2022.3\t总经理助理','北京猫眼国际货运代理有限公司','根据公司的经营理念和发展战略制定公司各工作岗位的工作目标;负责制定及推进公司员工的培训、绩效、薪酬的管理。','2020.2-2021.2\t运营助理','北京猫眼文化传媒有限公司','根据目标管理综合考评内容，深入、细致地到所跟踪的各场所了解、掌握情况，帮助和促进各场所提高行政、后勤管理水平。']
sentences2 = ['王美珠', '求职目标：市场总监-专注品牌方向', '王美珠', '求职目标：市场总监-专注品牌方向', '出生：1989.05', '住址：湖北省武汉市', '电话：13800138000', '邮箱：service@500d.me', '出生：1989.05', '住址：湖北省武汉市', '电话：13800138000', '邮箱：service@500d.me', '教育背景', '2014.07-2019.06                        湖南师范大学                  市场营销（博士）', '主修课程：管理学、微观经济学、宏观经济学、管理信息系统、统计学、会计学、财务管理、市场营销、经济法、消费者行为学、国际市场营销', '20011.07-2014.06                        湖南师范大学                  市场营销（硕士）', '主修课程：管理学、微观经济学、宏观经济学、管理信息系统、统计学、会计学、财务管理、市场营销、经济法、消费者行为学、国际市场营销', '教育背景', '2014.07-2019.06                        湖南师范大学                  市场营销（博士）', '主修课程：管理学、微观经济学、宏观经济学、管理信息系统、统计学、会计学、财务管理、市场营销、经济法、消费者行为学、国际市场营销', '20011.07-2014.06                        湖南师范大学                  市场营销（硕士）', '主修课程：管理学、微观经济学、宏观经济学、管理信息系统、统计学、会计学、财务管理、市场营销、经济法、消费者行为学、国际市场营销', '工作经历', '2022.09-至今                           辉世设计控股集团                  副总监', '负责协助集团旗下事业部开展各项工作，制定品牌传播方案；', '结合集团与事业部发展，制定营销策略、广告策略、品牌策略和公关策略，并组织推进执行；', '制定和执行媒体投放计划，跟踪和监督媒体投放效果，进行数据分析与撰写报告；', '负责公司品牌形象和价值提升的持续优化，提高品牌知名度；', '研究行业发展动态，定期进行市场调查,为产品更新提供建议。', '工作经历', '2022.09-至今                           辉世设计控股集团                  副总监', '负责协助集团旗下事业部开展各项工作，制定品牌传播方案；', '结合集团与事业部发展，制定营销策略、广告策略、品牌策略和公关策略，并组织推进执行；', '制定和执行媒体投放计划，跟踪和监督媒体投放效果，进行数据分析与撰写报告；', '负责公司品牌形象和价值提升的持续优化，提高品牌知名度；', '研究行业发展动态，定期进行市场调查,为产品更新提供建议。', '校园经历', '2011.09-2012.06                        湖南师范大学                  辩论队（队长）', '任职描述：', '负责50余人团队的日常训练、选拔及团队建设；', '作为负责人对接多项商业校园行活动，如《奔跑吧兄弟》大学站录制、长安福特全球校园行、《时代周末》校园行；', '代表湖南师范大学辩论队参与多项国际型比赛，并取得多项荣誉：', '湖南师范大学“唇舌烽火”辩论赛冠军、亚军；', '湖南师范大学2011级“杰出辩手”；', '2012.11-2014.06                        沟通与交流协会                  创始人/副会长', '任职描述：', '协助湖南省沟通协会湖南师范大学分部，从零开始组建初期团队；', '策划协会会员制，选拔、培训协会导师，推出一系列沟通课程；配合其他社团组织，承办诸如演讲、主持、讲坛等省级/校级活动；', '校园经历', '2011.09-2012.06                        湖南师范大学                  辩论队（队长）', '任职描述：', '负责50余人团队的日常训练、选拔及团队建设；', '作为负责人对接多项商业校园行活动，如《奔跑吧兄弟》大学站录制、长安福特全球校园行、《时代周末》校园行；', '代表湖南师范大学辩论队参与多项国际型比赛，并取得多项荣誉：', '湖南师范大学“唇舌烽火”辩论赛冠军、亚军；', '湖南师范大学2011级“杰出辩手”；', '2012.11-2014.06                        沟通与交流协会                  创始人/副会长', '任职描述：', '协助湖南省沟通协会湖南师范大学分部，从零开始组建初期团队；', '策划协会会员制，选拔、培训协会导师，推出一系列沟通课程；配合其他社团组织，承办诸如演讲、主持、讲坛等省级/校级活动；', '自我评价', '拥有多年的市场管理及品牌营销经验，卓越的规划、组织、策划、方案执行和团队领导能力，积累较强的人际关系处理能力和商务谈判技巧，善于沟通，具备良好的合作关系掌控能力与市场开拓能力；敏感的商业和市场意识，具备优秀的资源整合能力、业务推进能力； 思维敏捷，有培训演讲能力，懂激励艺术，能带动团队的积极性；擅长协调平衡团队成员的竞争与合作的关系，善于通过培训提高团队综合能力和凝聚力。',
              '自我评价', '拥有多年的市场管理及品牌营销经验，卓越的规划、组织、策划、方案执行和团队领导能力，积累较强的人际关系处理能力和商务谈判技巧，善于沟通，具备良好的合作关系掌控能力与市场开拓能力；敏感的商业和市场意识，具备优秀的资源整合能力、业务推进能力； 思维敏捷，有培训演讲能力，懂激励艺术，能带动团队的积极性；擅长协调平衡团队成员的竞争与合作的关系，善于通过培训提高团队综合能力和凝聚力。', '掌握技能', '普通话一级甲等', '通过全国计算机二级考试，熟练运用office相关软件。', '熟练使用绘声绘色软件，剪辑过各种类型的电影及班级视频。', '大学英语四/六级（CET-4/6），良好听说读写能力，快速浏览英语专业书籍。', '掌握技能', '普通话一级甲等', '通过全国计算机二级考试，熟练运用office相关软件。', '熟练使用绘声绘色软件，剪辑过各种类型的电影及班级视频。', '大学英语四/六级（CET-4/6），良好听说读写能力，快速浏览英语专业书籍。', '项目经历', '信健设计集团品牌升级发布会', '集团全新品牌logo及VI上线，在多渠道进行了传播；', '企业VIP客户群体逾60人，结合了线上发布、线下体验；', '后续媒体报道持续升温，子品牌罄玉结合代言人罗嘉良制造话题营销，为期3周；', '鼎基设计商业模式发布会', '整场活动以会议+洽谈双重模式进行，首日以介绍鼎基内部平台资源优势，政府背景优势等为主，一对多推介会进行推广普及；', '现场签署地方合作意向书，如：新疆、江西、浙江等优秀企业商户；', '以中国的波尔多为宣传点，主推旗下新疆大型项目，罄玉生态葡萄庄园、沙漠主题俱乐部等，制造营销、品牌热点。', '项目经历', '信健设计集团品牌升级发布会', '集团全新品牌logo及VI上线，在多渠道进行了传播；', '企业VIP客户群体逾60人，结合了线上发布、线下体验；', '后续媒体报道持续升温，子品牌罄玉结合代言人罗嘉良制造话题营销，为期3周；', '鼎基设计商业模式发布会', '整场活动以会议+洽谈双重模式进行，首日以介绍鼎基内部平台资源优势，政府背景优势等为主，一对多推介会进行推广普及；', '现场签署地方合作意向书，如：新疆、江西、浙江等优秀企业商户；', '以中国的波尔多为宣传点，主推旗下新疆大型项目，罄玉生态葡萄庄园、沙漠主题俱乐部等，制造营销、品牌热点。', '2021.09-2022.09                        鼎基设计有限公司                 市场及运营总监', '根据公司发展情况进行战略调整，配合前端销售部门搭建销售渠道；', '负责公司品牌形象和价值提升的持续优化，提高品牌知名度；', '研究行业发展动态，定期进行市场调查,为产品更新提供建议；', '负责公司部门(营运、品牌策划)制度规范，负责组织及监管市场部关于对外合作、渠道管理、媒体合作、推广策划以相关工作的落实。', '2020.09-2021.09                        宏建设计俱乐部                   市场副总监', '负责事业部产品对外推广和宣传，制定各种整合营销的活动；', '执行媒体投放计划，跟踪和监督媒体投放效果，进行数据分析撰写报告；', '向市场总监提供营销支持，并协助相关的公关事宜。', '2019.09-2020.09                        信健设计集团                     市场部-市场经理', '负责集团SEO优化事宜，在未使用百度竞价排名服务前提下，连接占据首页4至12条；', '参与策划集团年度、季度、月度线下推广活动方案；', '负责线下大型分享会”思享潭“的执行与后期线上推广。', '2021.09-2022.09                        鼎基设计有限公司                 市场及运营总监', '根据公司发展情况进行战略调整，配合前端销售部门搭建销售渠道；', '负责公司品牌形象和价值提升的持续优化，提高品牌知名度；', '研究行业发展动态，定期进行市场调查,为产品更新提供建议；', '负责公司部门(营运、品牌策划)制度规范，负责组织及监管市场部关于对外合作、渠道管理、媒体合作、推广策划以相关工作的落实。', '2020.09-2021.09                        宏建设计俱乐部                   市场副总监', '负责事业部产品对外推广和宣传，制定各种整合营销的活动；', '执行媒体投放计划，跟踪和监督媒体投放效果，进行数据分析撰写报告；', '向市场总监提供营销支持，并协助相关的公关事宜。', '2019.09-2020.09                        信健设计集团                     市场部-市场经理', '负责集团SEO优化事宜，在未使用百度竞价排名服务前提下，连接占据首页4至12条；', '参与策划集团年度、季度、月度线下推广活动方案；', '负责线下大型分享会”思享潭“的执行与后期线上推广。']
sentences3 = ['蔡依婷', '蔡依婷', '姓名：蔡依婷', '30岁', '籍贯：上海', '民族：汉族', '学历：本科', '专业：设计', '姓名：蔡依婷', '年龄：30岁', '籍贯：上海', '民族：汉族', '学历：本科', '专业：设计', '求 职 意 向 ： 设 计 师', '求 职 意 向 ： 设 计 师', '联系方式', '联系方式', '获得证书', '获得证书', '2011 - 2015', '本科', '2011 - 2015', '本科', '工作经历', '工作经历', 'Adobe认证设计师', 'Adobe认证产品专家', '平面设计师资格证书', '平面设计师资格证书', '英语四六级证书', 'Adobe认证设计师', 'Adobe认证产品专家', '平面设计师资格证书', '平面设计师资格证书', '英语四六级证书', '校园活动', '校园活动', '教育背景', '教育背景', '北京理工大学', '设计专业', '北京理工大学', '设计专业', '13912345678', 'xianxian@taobao.com', '13912345678', 'xianxian@taobao.com', '大学生三下乡', '在湖南省湘西自治州保靖县木芽村小学义务支教活动，被评为优秀支教老师。修改为自己活动内容介绍。修改为自己内容介绍。', '大学生三下乡', '在湖南省湘西自治州保靖县木芽村小学义务支教活动，被评为优秀支教老师。修改为自己活动内容介绍。修改为自己内容介绍。', '党支部书记', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。', '党支部书记', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。', '英语俱乐部干事', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。', '英语俱乐部干事', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。', '学生会主席', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。修改为自己活动内容介绍。', '学生会主席', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。修改为自己活动内容介绍。', '2015.9 – 2019.9', '上海', '2015.9 – 2019.9', '上海', '2019.9 – 至今', '上海', '2019.9 – 至今', '上海', '智扬顾问有限公司', '平面设计师', '智扬顾问有限公司', '平面设计师', '负责企业画册设计、标志设计、产品包装设计、杂志广告跨页设计、户外广告', '对公司的网站后台周期性的修改和维护', '对公司所有的宣传广告全程负责一直到完成阶段', '负责企业画册设计、标志设计、产品包装设计、杂志广告跨页设计、户外广告', '对公司的网站后台周期性的修改和维护', '对公司所有的宣传广告全程负责一直到完成阶段', '美铭顾问有限公司', '平面设计师', '美铭顾问有限公司', '平面设计师', '负责企业画册设计、标志设计、产品包装设计、杂志广告跨页设计、户外广告', '对公司的网站后台周期性的修改和维护', '对公司所有的宣传广告全程负责一直到完成阶段',
              '负责企业画册设计、标志设计、产品包装设计、杂志广告跨页设计、户外广告', '对公司的网站后台周期性的修改和维护', '对公司所有的宣传广告全程负责一直到完成阶段', '求 职 意 向 ： 设 计 师', '求 职 意 向 ： 设 计 师', '蔡依婷', '蔡依婷', '13912345678', 'xianxian@taobao.com', '13912345678', 'xianxian@taobao.com', '姓名：奈森设计', '年龄：24岁', '籍贯：上海', '民族：汉族', '学历：本科', '专业：设计', '姓名：奈森设计', '年龄：24岁', '籍贯：上海', '民族：汉族', '学历：本科', '专业：设计', '联系方式', '联系方式', '个人爱好', '个人爱好', '自我评估', '自我评估', '爬山', '写作', '音乐', '瑜伽', '爬山', '写作', '音乐', '瑜伽', '独立负责过大型项目，非常成功的规划了项目 A 工作；', 'B 项目，从开始到完成，极大地加强了我的逻辑思考能力；', '具备艺术家的灵感创作能力，公司公认的首席 PPT 制作人；', '非常优秀的用户心理洞察力，配合丰富的创意及文案能力极大地提高了我的执行力。段前间距设为0.2行。', '专业知识的学习以及多年的工作实践，使我积累了丰富的工作经验，并取得了优秀的销售业绩。段前间距设为0.2行。', '独立负责过大型项目，非常成功的规划了项目 A 工作；', 'B 项目，从开始到完成，极大地加强了我的逻辑思考能力；', '具备艺术家的灵感创作能力，公司公认的首席 PPT 制作人；', '非常优秀的用户心理洞察力，配合丰富的创意及文案能力极大地提高了我的执行力。段前间距设为0.2行。', '专业知识的学习以及多年的工作实践，使我积累了丰富的工作经验，并取得了优秀的销售业绩。段前间距设为0.2行。', 'MS Office', 'Photoshop', 'Illustrator', '英语', '日语', '粤语', 'MS Office', 'Photoshop', 'Illustrator', '英语', '日语', '粤语', '2011年国家励志一等奖学金', '2012年北京理工大学挑战主持人大赛第一名', '2012年北京理工大学演讲比赛三等奖', '2013年全国大学生数学建模大赛国家二等奖', '2013年担任英语俱乐部主席期间，领导英语俱乐部获得“某年度十佳社团”荣誉称号', '2014年北京理工大学第九届数学建模大赛一等奖', '2015年参加省红十字协会与校团委主办的演讲比赛并取得前三名的佳绩（全校近200人参赛）', '2011年国家励志一等奖学金', '2012年北京理工大学挑战主持人大赛第一名', '2012年北京理工大学演讲比赛三等奖', '2013年全国大学生数学建模大赛国家二等奖', '2013年担任英语俱乐部主席期间，领导英语俱乐部获得“某年度十佳社团”荣誉称号', '2014年北京理工大学第九届数学建模大赛一等奖', '2015年参加省红十字协会与校团委主办的演讲比赛并取得前三名的佳绩（全校近200人参赛）', '精通', '熟练', '熟练', '流利', '一般', '流利', '精通', '熟练', '熟练', '流利', '一般', '流利', '掌握技能', '掌握技能', '获奖经历', '获奖经历']
print(analysis(sentences1))
print(analysis(sentences2))
print(analysis(sentences3))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{"basic_data": {"name": "张吉惟", "birth": "1998.11", "age": 26, "tel": "138 9999 9999", "email": "zhangjiwei@163.com", "college": ["北京师范大学"], "loc": ["北京", "海淀区"], "edu": "本科"}, "job_obj": ["行政管理类"], "experience": ["2022.3至今\t部长助理", "北京猫眼机械有限公司", "质管部部长助理.负责票证车间生产质量检查,控制,兼任企业内审员,推行ISO9001:2008及ISO14000体系并年审。", "2021.2-2022.3\t总经理助理", "北京猫眼国际货运代理有限公司", "根据公司的经营理念和发展战略制定公司各工作岗位的工作目标;负责制定及推进公司员工的培训、绩效、薪酬的管理。", "2020.2-2021.2\t运营助理", "北京猫眼文化传媒有限公司", "根据目标管理综合考评内容，深入、细致地到所跟踪的各场所了解、掌握情况，帮助和促进各场所提高行政、后勤管理水平。"], "award": ["2016\t全国大学生英语竞赛一等奖", "2017\t国家级一等奖学金"], "ability": ["EXCEL", "PPT", "WORD", "AI", "PS", "2018\t大学英语6级证书（CET-6）"], "job_fit": ["人力资源管理", "文员"], "tag": {"edu_tag": ["985", "本科"], "loc_tag": "北京", "experience_tag": ["北京猫眼机械有限公司", "北京猫眼国际货运代理有限公司", "北京猫眼文化传媒有限公司"], "ability": ["Office", "Photoshop", "CET6"], "total_work_time": 4}, "score": 27.3, "custom_content": {"money_obj": "", "self_desc": [], "self_tag": []}}
{"basic_data": {"name": "王美珠", "birth": "1989.05", "age": 35, "tel"